In [1]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [2]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class4.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [3]:
def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
for grade in grades: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                incorrect_id = value[0]
                print("Invalid line of data:", (incorrect_id) + " does not contain exactly 26 values:")
                print(grade)    
values_count = list(map(valueCheck,grades))
values_check = list(map(has26Values,values_count))
print("Total valid lines of data:", values_check.count(True))
print("Total invalid lines of data:", values_check.count(False))


Invalid line of data: N00000616 does not contain exactly 26 values:
N00000616,B,A,D,D,A,B,D,A,C,C,D,C,A,B,A,C,B,D,A,,A,B,B,D,D,A

Invalid line of data: N00000638 does not contain exactly 26 values:
N00000638,B,A,D,D,C,B,D,A,C,,D,B,A,B,A,B,C,D,A,C,A,,,D,D,Q

Total valid lines of data: 493
Total invalid lines of data: 2


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html
https://linuxhint.com/pandas-compare-two-dataframes-row-by-row/
pandas.drop 
https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bool.html 
https://stackoverflow.com/questions/35307732/how-to-sum-in-pandas-by-unique-index-in-several-columns
https://stackoverflow.com/questions/44564414/how-can-i-use-return-to-get-back-multiple-values-from-a-loop-can-i-put-them-i 

--------------------
How to get the incorrect answer to a new dataframe ( make a new dataframe with the quantity of answering incorrectly, score

In [25]:
from symbol import comparison


answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
#print(answer_key_table)
def checkAnswer(grades, answer_key_table):
        for grade in grades: 
                value = grade.split(",")
                answer_table = pd.DataFrame(value)
                answer_table = answer_table.iloc[1: , :]
                answer_table = answer_table.reset_index(drop=True)
                cVal = answer_table.iloc[24][0]
                cValSplit = cVal.split("\n")
                c = cValSplit[0]
                answer_table = answer_table.replace([cVal],c)
                print(answer_table)
                comparison = answer_key_table.compare(answer_table,align_axis=0)
                #print(answer_key_table)
                #comparison_table = pd.DataFrame(comparison)
                #gradeCheck = comparison_table.value_counts(dropna=False)
                #gradeCheck.to_frame()
                #score  = gradeCheck[0]
                #correctAnswer = score*2
                #a = comparison.loc[comparison['other'] == 'A']
                #comparison['Address'] = 0
                #comparison["Address"] = comparison["Address"] + 1
                #a = comparison.groupby(level=0)["Address"].sum()
                #print(a)
        #return comparison
        #print(comparison)    
#df = df.groupby('id').sum()
checkAnswer(grades, answer_key_table)


    0
0   B
1   A
2    
3   D
4   C
5   B
6   D
7   A
8   C
9   C
10  D
11  B
12  A
13  B
14  A
15  C
16  B
17  D
18  C
19  C
20  A
21  A
22  B
23  D
24  A
    0
0   B
1    
2   D
3   C
4    
5   B
6   D
7   A
8   C
9    
10  D
11  B
12  A
13  B
14  B
15  C
16  B
17  D
18  A
19  C
20  A
21  A
22  B
23  D
24  D
    0
0   B
1   A
2   D
3   D
4   C
5   B
6   D
7   A
8   C
9   B
10  D
11  B
12  A
13  B
14  A
15  C
16  B
17  D
18  D
19  C
20  A
21  A
22  B
23  D
24  D
    0
0   B
1   A
2   D
3   D
4   C
5   B
6   B
7   A
8   C
9   C
10  A
11  B
12   
13  B
14  A
15  C
16  B
17  D
18   
19  C
20  A
21  A
22  B
23   
24  D
    0
0   B
1   A
2   D
3   D
4    
5   B
6   D
7   A
8   C
9   C
10   
11  B
12  C
13  B
14  D
15  C
16  B
17   
18  B
19  C
20  B
21  A
22  B
23  D
24  D
    0
0   B
1    
2   D
3   D
4   C
5   B
6   D
7   A
8   C
9   C
10  D
11  B
12  A
13  B
14  A
15  C
16   
17  D
18  A
19  C
20  A
21   
22  A
23  D
24   
    0
0   B
1   A
2   D
3   D
4   C
5   B
6   D
7    
8   C
9   

ValueError: Can only compare identically-labeled DataFrame objects

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'